# Audio analysis with TimeSide and [Aubio](http://aubio.org)

In the following example, we illustrate how to perform common audio signal analysis tasks like *pitch estimation* and *onsets detection* with **TimeSide** by using some analyzers based on [Aubio](http://aubio.org).

In [ ]:
%pylab inline

from __future__ import division
import matplotlib.pylab as pylab
pylab.rcParams['figure.figsize'] = 16, 8  # that's default image size for this interactive session

import timeside
from timeside.core import get_processor

import matplotlib.pyplot as plt
import numpy as np

# Audio Source = a 15s excerpt of Peter and the Wolf by Prokofiev
# from the Internet Archive : https://archive.org/details/PeterAndTheWolf_753
audiofile = 'https://ia801408.us.archive.org/3/items/PeterAndTheWolf_753/Peter_and_the_Wolf.mp3'

## Setup the processing pipe
file_decoder = get_processor('file_decoder')(uri=audiofile, start=5, duration=15)
aubio_pitch = get_processor('aubio_pitch')()
aubio_temporal = get_processor('aubio_temporal')()
specgram_ = get_processor('spectrogram_analyzer')()
waveform = get_processor('waveform_analyzer')()


pipe = (file_decoder | aubio_pitch | aubio_temporal | specgram_ | waveform)
pipe.run()

## Display Spectrogram + Aubio Pitch + Aubio Beat

In [ ]:
plt.figure(1)

spec_res = specgram_.results['spectrogram_analyzer']
N = spec_res.parameters['fft_size']
plt.imshow(20 * np.log10(spec_res.data.T),
           origin='lower',
           extent=[spec_res.time[0], spec_res.time[-1], 0,
                   (N // 2 + 1) / N * spec_res.data_object.frame_metadata.samplerate],
           aspect='auto')

res_pitch = aubio_pitch.results['aubio_pitch.pitch']
plt.plot(res_pitch.time, res_pitch.data)

res_beats = aubio_temporal.results['aubio_temporal.beat']

for time in res_beats.time:
    plt.axvline(time, color='r')

plt.title('Spectrogram + Aubio pitch + Aubio beat')
plt.grid()

## Display waveform + Onsets

In [ ]:
plt.figure(2)
res_wave = waveform.results['waveform_analyzer']
plt.plot(res_wave.time, res_wave.data)
res_onsets = aubio_temporal.results['aubio_temporal.onset']
for time in res_onsets.time:
    plt.axvline(time, color='r')
plt.grid()
plt.title('Waveform + Aubio onset')
plt.show()

In [ ]:
res_pitch.render();

In [ ]:
res_beats.render();

In [ ]:
res_onsets.render();